In [ ]:
import pandas as pd
import nltk
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from ragas import evaluate
from ragas.metrics import (
    answer_relevancy,
    faithfulness,
    answer_correctness,
    answer_similarity
)
from datasets import Dataset
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
import numpy as np

# Download NLTK data
nltk.download('punkt')

# Initialize LLM for RAGAS (you can use OpenAI or other providers)
llm = ChatOpenAI(model="gpt-4", temperature=0)
embeddings = OpenAIEmbeddings()

def calculate_bleu_score(reference, candidate):
    """Calculate BLEU score between reference and candidate answers"""
    try:
        reference_tokens = reference.lower().split()
        candidate_tokens = candidate.lower().split()
        
        # Use smoothing function to avoid zero scores
        smoothie = SmoothingFunction().method4
        score = sentence_bleu([reference_tokens], candidate_tokens, 
                             smoothing_function=smoothie)
        return round(score, 4)
    except:
        return 0.0

def evaluate_answers(excel_file, sheet_name='Sheet1'):
    """
    Evaluate answers from Excel sheet with multiple metrics
    
    Parameters:
    -----------
    excel_file : str
        Path to Excel file
    sheet_name : str
        Name of the sheet to read
    """
    
    # Read Excel file
    df = pd.read_excel(excel_file, sheet_name=sheet_name)
    
    print(f"Loaded {len(df)} rows from Excel")
    print(f"Columns: {df.columns.tolist()}")
    
    # Initialize result columns
    df['BLEU_GPT_vs_Human'] = 0.0
    df['BLEU_OSS_vs_Human'] = 0.0
    df['BLEU_RAG_vs_Human'] = 0.0
    
    # Calculate BLEU scores
    print("\nCalculating BLEU scores...")
    for idx, row in df.iterrows():
        human_answer = str(row['human_answer'])
        
        if 'gpt_answer' in df.columns:
            df.at[idx, 'BLEU_GPT_vs_Human'] = calculate_bleu_score(
                human_answer, str(row['gpt_answer'])
            )
        
        if 'oss_answer' in df.columns:
            df.at[idx, 'BLEU_OSS_vs_Human'] = calculate_bleu_score(
                human_answer, str(row['oss_answer'])
            )
        
        if 'rag_answer' in df.columns:
            df.at[idx, 'BLEU_RAG_vs_Human'] = calculate_bleu_score(
                human_answer, str(row['rag_answer'])
            )
    
    # Prepare data for RAGAS evaluation
    print("\nPreparing data for RAGAS evaluation...")
    
    # For GPT answers
    if 'gpt_answer' in df.columns and 'question' in df.columns:
        gpt_dataset = Dataset.from_dict({
            'question': df['question'].tolist(),
            'answer': df['gpt_answer'].tolist(),
            'ground_truth': df['human_answer'].tolist(),
            'contexts': [[str(ans)] for ans in df['gpt_answer'].tolist()]
        })
        
        print("\nEvaluating GPT answers with RAGAS...")
        gpt_results = evaluate(
            gpt_dataset,
            metrics=[answer_correctness, answer_similarity, answer_relevancy],
            llm=llm,
            embeddings=embeddings
        )
        
        gpt_df = gpt_results.to_pandas()
        df['RAGAS_GPT_Correctness'] = gpt_df['answer_correctness'].values
        df['RAGAS_GPT_Similarity'] = gpt_df['answer_similarity'].values
        df['RAGAS_GPT_Relevancy'] = gpt_df['answer_relevancy'].values
    
    # For OSS answers
    if 'oss_answer' in df.columns and 'question' in df.columns:
        oss_dataset = Dataset.from_dict({
            'question': df['question'].tolist(),
            'answer': df['oss_answer'].tolist(),
            'ground_truth': df['human_answer'].tolist(),
            'contexts': [[str(ans)] for ans in df['oss_answer'].tolist()]
        })
        
        print("\nEvaluating OSS answers with RAGAS...")
        oss_results = evaluate(
            oss_dataset,
            metrics=[answer_correctness, answer_similarity, answer_relevancy],
            llm=llm,
            embeddings=embeddings
        )
        
        oss_df = oss_results.to_pandas()
        df['RAGAS_OSS_Correctness'] = oss_df['answer_correctness'].values
        df['RAGAS_OSS_Similarity'] = oss_df['answer_similarity'].values
        df['RAGAS_OSS_Relevancy'] = oss_df['answer_relevancy'].values
    
    # For RAG answers
    if 'rag_answer' in df.columns and 'question' in df.columns:
        rag_dataset = Dataset.from_dict({
            'question': df['question'].tolist(),
            'answer': df['rag_answer'].tolist(),
            'ground_truth': df['human_answer'].tolist(),
            'contexts': [[str(ans)] for ans in df['rag_answer'].tolist()]
        })
        
        print("\nEvaluating RAG answers with RAGAS...")
        rag_results = evaluate(
            rag_dataset,
            metrics=[answer_correctness, answer_similarity, answer_relevancy, faithfulness],
            llm=llm,
            embeddings=embeddings
        )
        
        rag_df = rag_results.to_pandas()
        df['RAGAS_RAG_Correctness'] = rag_df['answer_correctness'].values
        df['RAGAS_RAG_Similarity'] = rag_df['answer_similarity'].values
        df['RAGAS_RAG_Relevancy'] = rag_df['answer_relevancy'].values
        df['RAGAS_RAG_Faithfulness'] = rag_df['faithfulness'].values
    
    # Calculate average scores
    print("\nCalculating summary statistics...")
    summary = {
        'Metric': [],
        'GPT_Avg': [],
        'OSS_Avg': [],
        'RAG_Avg': []
    }
    
    metrics = ['BLEU', 'RAGAS_Correctness', 'RAGAS_Similarity', 'RAGAS_Relevancy']
    
    for metric in metrics:
        summary['Metric'].append(metric)
        
        gpt_col = f"{metric}_GPT_vs_Human" if metric == 'BLEU' else f"{metric.replace('RAGAS_', 'RAGAS_GPT_')}"
        oss_col = f"{metric}_OSS_vs_Human" if metric == 'BLEU' else f"{metric.replace('RAGAS_', 'RAGAS_OSS_')}"
        rag_col = f"{metric}_RAG_vs_Human" if metric == 'BLEU' else f"{metric.replace('RAGAS_', 'RAGAS_RAG_')}"
        
        summary['GPT_Avg'].append(round(df[gpt_col].mean(), 4) if gpt_col in df.columns else 'N/A')
        summary['OSS_Avg'].append(round(df[oss_col].mean(), 4) if oss_col in df.columns else 'N/A')
        summary['RAG_Avg'].append(round(df[rag_col].mean(), 4) if rag_col in df.columns else 'N/A')
    
    summary_df = pd.DataFrame(summary)
    
    # Save results
    output_file = excel_file.replace('.xlsx', '_evaluated.xlsx')
    with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
        df.to_excel(writer, sheet_name='Detailed_Results', index=False)
        summary_df.to_excel(writer, sheet_name='Summary', index=False)
    
    print(f"\n✅ Results saved to: {output_file}")
    print("\nSummary Statistics:")
    print(summary_df.to_string(index=False))
    
    return df, summary_df

# Usage
if __name__ == "__main__":
    # Update with your file path and column names
    excel_file = "your_file.xlsx"
    
    # Make sure your Excel has these columns:
    # - question
    # - human_answer
    # - gpt_answer
    # - oss_answer
    # - rag_answer
    
    results_df, summary = evaluate_answers(excel_file)
